# V2.1 RAG with Strategy Template
Testing the effects of model temperature and other possible factors on LLM output generation.

One of the key issue identified in V2.1 is the variability of output generated by the LLM. As displayed in the sample output in folder `v2.0_sample_output`, the code generated by the LLM model and even the retriever results can change, given the same ingested data and text splitting specification.

Hence, in this version, the focus will be on investigating factors that affects LLM output randomness and how to stabilize them.

We will use the arguments: \
```chunk_size = 2000``` \
```chunk_overlap = 500```\
for the text splitter as it was tested and believed to be the better performing set of values from `v2.0_rag_strategy_template`.

## Takeaways
https://medium.com/@prabhavithreddy/controlling-creativity-how-to-get-reproducible-outcomes-from-llms-016ec0991891

1. Model Temperature
    - Setting temperature = 0 achieves the best effect on generating outputs that are similar and determinant. 
    - The code generated by ```temperature=0``` are almost identical, while the code explanation can vary across responses.
2. Seed 
    - Setting a specific does not show obvious effects on improving reproducibility. Moreover, it is hard to define which seed is better. 
    - Rather than spending time identifying the "optimal" seed, we should spend time on enhancing other components of the pipeline for better performance.

In [1]:
%pip install --user -qU langchain
%pip install --user -qU langchain_community
%pip install --user -qU langchain_chroma
%pip install --user -qU langchain-openai
%pip install --user -qU langchainhub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Model Temperature
Configuring the model temperature is one of the first we can do to control the randomness of LLM responses.

The default temperature set for the OpenAI gpt3.5 is 0.7. This configuration was used in previous versions.

For this section, we will be reusing the same system prompt and vector store (same embeddings)

In [2]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PythonLoader
from langchain_text_splitters import ( Language, RecursiveCharacterTextSplitter)

In [4]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

### Temperature = 0.7 (default)

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.7)

In [6]:
folder_path = "strategy-template"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = PythonLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [7]:
print(documents[0])

page_content='class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 45 * 60
        self.options = {}

        self.divide_quote = 0
        self.proportion = 0.2
        self.aroon_period = 14

    def on_order_state_change(self,  order):
        pass

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]
        open_price_history = [candle['open'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()
        open_price_history.reverse()

        # convert to np.

In [8]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=2000, 
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [9]:
# View the chunks that has the source of 'RSI-en.py'
rsi_chunks = [chunk for chunk in text_chunks if chunk.metadata['source'] == 'RSI-en.py']
for chunk in rsi_chunks:
    print(chunk.metadata)

{'source': 'RSI-en.py', 'start_index': 0}
{'source': 'RSI-en.py', 'start_index': 1302}
{'source': 'RSI-en.py', 'start_index': 2626}
{'source': 'RSI-en.py', 'start_index': 4201}


In [10]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [11]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [12]:
response_1 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_1["answer"])

To create a strategy where you buy Bitcoin if its RSI drops below 30 and sell if it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the modified code snippet for your strategy:

```python
class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.short_period = 14  # RSI short period
        self.long_period = 28  # RSI long period
        self.proportion = 0.2  # Proportion for trade amount
        self.rsi_lower_band = 30  # RSI lower band for buying
        self.rsi_upper_band = 70  # RSI upper band for selling
        self.trade_amount = 0  # Amount to trade

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()

        close_price_history = [candle['close'] for candle in candles[exchange][pair]]

        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history)

In [13]:
for document in response_1["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [14]:
response_2 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_2["answer"])

To create a strategy where you buy Bitcoin if its RSI drops below 30 and sell if it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version that focuses on the buying and selling based on RSI levels:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.rsi_period = 14
        self.rsi_upper_threshold = 70
        self.rsi_lower_threshold = 30
        self.in_position = False

    def trade(self, close_price_history):
        rsi = talib.RSI(np.array(close_price_history), timeperiod=self.rsi_period)
        curr_rsi = rsi[-1]

        signal = 0
        if curr_rsi < self.rsi_lower_threshold and not self.in_position:
            signal = 1  # Buy signal
            self.in_position = True
        elif curr_rsi > self.rsi_upper_threshold and self.in_position:
            signal = -1  # Sell signal
            self.in_position = False

        return signal

# Example of how to

In [15]:
for document in response_2["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [16]:
response_3 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_3["answer"])

To create a strategy where you buy Bitcoin if its Relative Strength Index (RSI) drops below 30 and sell if it goes above 70, you can modify the existing code snippet provided from Crypto Arsenal's documentation. Here's an outline of the modifications you can make:

1. Update the RSI upper and lower bands to 70 and 30 respectively.
2. Adjust the signal logic to buy when RSI drops below 30 and sell when it goes above 70.

Here is the modified code snippet based on the existing code structure:

```python
class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 30 * 60
        self.options = {}

        self.last_type = 'sell'
        self.short_period = 5
        self.long_period = 10
        self.divide_quote = 0
        self.proportion = 0.2

        self.rsi_upper_band = 70  # Updated upper RSI band to 70
        self.rsi_lower_band = 30  # Updated lower RSI band to 30

    def trade(self, candles):
     

In [17]:
for document in response_3["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


### Temperature = 0.4

In [18]:
# Reseting Chroma vectorstore collection
vectorstore.reset_collection()

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.4)

In [20]:
folder_path = "strategy-template"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = PythonLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [21]:
print(documents[0])

page_content='class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 45 * 60
        self.options = {}

        self.divide_quote = 0
        self.proportion = 0.2
        self.aroon_period = 14

    def on_order_state_change(self,  order):
        pass

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]
        open_price_history = [candle['open'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()
        open_price_history.reverse()

        # convert to np.

In [22]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=2000, 
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [23]:
# View the chunks that has the source of 'RSI-en.py'
rsi_chunks = [chunk for chunk in text_chunks if chunk.metadata['source'] == 'RSI-en.py']
for chunk in rsi_chunks:
    print(chunk.metadata)

{'source': 'RSI-en.py', 'start_index': 0}
{'source': 'RSI-en.py', 'start_index': 1302}
{'source': 'RSI-en.py', 'start_index': 2626}
{'source': 'RSI-en.py', 'start_index': 4201}


In [24]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [25]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [26]:
response_1 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_1["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code snippet provided from Crypto Arsenal's documentation. Here's a modified version of the code snippet:

```python
import numpy as np
import talib

class RsiStrategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.short_period = 14
        self.long_period = 28
        self.rsi_lower_band = 30
        self.rsi_upper_band = 70

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)
        
        if len(close_price_history) < self.long_period + 1:
            return []
        
        curr_rsi_short = rs

In [27]:
for document in response_1["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [28]:
response_2 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_2["answer"])

To create a strategy where you buy Bitcoin if its RSI drops below 30 and sell if it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy based on your requirements:

```python
import numpy as np
import talib

class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.short_period = 14  # RSI short period
        self.long_period = 28  # RSI long period
        self.rsi_lower_band = 30  # RSI lower band for buying
        self.rsi_upper_band = 70  # RSI upper band for selling

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()

        close_price_history = [candle['close'] for candle in candles[exchange][pair]]

        # Calculate RSI
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        # Current RSI values
       

In [29]:
for document in response_2["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [30]:
response_3 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_3["answer"])

To create a strategy where you buy Bitcoin if its RSI drops below 30 and sell if it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here is a simplified version of the strategy based on the requirements:

```python
import talib
import numpy as np

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_lower_band = 30
        self.rsi_upper_band = 70
        self.divide_quote = 0

    def trade(self, close_price_history):
        rsi_short = talib.RSI(close_price_history, self.short_period)
        rsi_long = talib.RSI(close_price_history, self.long_period)

        if len(close_price_history) < self.long_period + 1:
            return []

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        signal = 0

        if curr_rsi_short < self.rsi_lower_band:
            signal = 1  # Buy signal
        elif curr_rsi_short > self.rsi_upper_band:
       

In [31]:
for document in response_3["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


### Temperature = 0

In [32]:
# Reseting Chroma vectorstore collection
vectorstore.reset_collection()

In [33]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [34]:
folder_path = "strategy-template"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = PythonLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [35]:
print(documents[0])

page_content='class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 45 * 60
        self.options = {}

        self.divide_quote = 0
        self.proportion = 0.2
        self.aroon_period = 14

    def on_order_state_change(self,  order):
        pass

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]
        open_price_history = [candle['open'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()
        open_price_history.reverse()

        # convert to np.

In [36]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=2000, 
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [37]:
# View the chunks that has the source of 'RSI-en.py'
rsi_chunks = [chunk for chunk in text_chunks if chunk.metadata['source'] == 'RSI-en.py']
for chunk in rsi_chunks:
    print(chunk.metadata)

{'source': 'RSI-en.py', 'start_index': 0}
{'source': 'RSI-en.py', 'start_index': 1302}
{'source': 'RSI-en.py', 'start_index': 2626}
{'source': 'RSI-en.py', 'start_index': 4201}


In [38]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [39]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [40]:
response_1 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_1["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_upper_band = 70
        self.rsi_lower_band = 30
        self.is_long = False

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band and not self.is_long:
            self.is_long = True
            return "Buy Bitcoin"

        if curr_rsi_short > self.rsi_upper_band and self.is_long:
        

In [41]:
for document in response_1["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [42]:
response_2 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_2["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_lower_band = 30
        self.rsi_upper_band = 70
        self.is_long = False

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band and not self.is_long:
            # Buy Bitcoin
            self.is_long = True
            return "Buy Bitcoin"

        elif curr_rsi_short > self.rsi_upper_ban

In [43]:
for document in response_2["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [44]:
response_3 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_3["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_upper_band = 70
        self.rsi_lower_band = 30
        self.is_long = False

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band and not self.is_long:
            self.is_long = True
            return "Buy Bitcoin"
        elif curr_rsi_short > self.rsi_upper_band and self.is_long:
       

In [45]:
for document in response_3["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


## Model Seed

### Seed = 100

In [52]:
# Reseting Chroma vectorstore collection
vectorstore.reset_collection()

In [53]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, seed=100)

In [54]:
folder_path = "strategy-template"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = PythonLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [55]:
print(documents[0])

page_content='class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 45 * 60
        self.options = {}

        self.divide_quote = 0
        self.proportion = 0.2
        self.aroon_period = 14

    def on_order_state_change(self,  order):
        pass

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]
        open_price_history = [candle['open'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()
        open_price_history.reverse()

        # convert to np.

In [56]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=2000, 
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [57]:
# View the chunks that has the source of 'RSI-en.py'
rsi_chunks = [chunk for chunk in text_chunks if chunk.metadata['source'] == 'RSI-en.py']
for chunk in rsi_chunks:
    print(chunk.metadata)

{'source': 'RSI-en.py', 'start_index': 0}
{'source': 'RSI-en.py', 'start_index': 1302}
{'source': 'RSI-en.py', 'start_index': 2626}
{'source': 'RSI-en.py', 'start_index': 4201}


In [58]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [59]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [60]:
response_1 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_1["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_upper_band = 70
        self.rsi_lower_band = 30
        self.is_long = False

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band and not self.is_long:
            self.is_long = True
            return "Buy Bitcoin"
        elif curr_rsi_short > self.rsi_upper_band and self.is_long:
       

In [61]:
for document in response_1["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [62]:
response_2 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_2["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_upper_band = 70
        self.rsi_lower_band = 30
        self.divide_quote = 0

    def trade(self, close_price_history):
        rsi_short = talib.RSI(close_price_history, self.short_period)
        rsi_long = talib.RSI(close_price_history, self.long_period)

        if len(close_price_history) < self.long_period + 1:
            return []

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        prev_rsi_short = rsi_short[-2]
        prev_rsi_long = rsi_long[-2]

        signal = 0

        if curr_rsi_short < self.rsi_lower_band a

In [63]:
for document in response_2["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [64]:
response_3 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_3["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_upper_band = 70
        self.rsi_lower_band = 30
        self.divide_quote = 0

    def trade(self, close_price_history):
        rsi_short = talib.RSI(close_price_history, self.short_period)
        rsi_long = talib.RSI(close_price_history, self.long_period)

        if len(close_price_history) < self.long_period + 1:
            return []

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band:
            # Buy Bitcoin
            return "Buy"

        if curr_rsi_short > self.rsi_upper_ba

In [65]:
for document in response_3["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


### Seed = 999

In [66]:
# Reseting Chroma vectorstore collection
vectorstore.reset_collection()

In [67]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, seed=999)

In [68]:
folder_path = "strategy-template"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = PythonLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [69]:
print(documents[0])

page_content='class Strategy(StrategyBase):
    def __init__(self):
        # strategy property
        self.subscribed_books = {}
        self.period = 45 * 60
        self.options = {}

        self.divide_quote = 0
        self.proportion = 0.2
        self.aroon_period = 14

    def on_order_state_change(self,  order):
        pass

    def trade(self, candles):
        exchange, pair, base, quote = CA.get_exchange_pair()
        
        close_price_history = [candle['close'] for candle in candles[exchange][pair]]
        high_price_history = [candle['high'] for candle in candles[exchange][pair]]
        low_price_history = [candle['low'] for candle in candles[exchange][pair]]
        open_price_history = [candle['open'] for candle in candles[exchange][pair]]

        # convert to chronological order for talib
        close_price_history.reverse()
        high_price_history.reverse()
        low_price_history.reverse()
        open_price_history.reverse()

        # convert to np.

In [70]:
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, 
    chunk_size=2000, 
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [71]:
# View the chunks that has the source of 'RSI-en.py'
rsi_chunks = [chunk for chunk in text_chunks if chunk.metadata['source'] == 'RSI-en.py']
for chunk in rsi_chunks:
    print(chunk.metadata)

{'source': 'RSI-en.py', 'start_index': 0}
{'source': 'RSI-en.py', 'start_index': 1302}
{'source': 'RSI-en.py', 'start_index': 2626}
{'source': 'RSI-en.py', 'start_index': 4201}


In [72]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [73]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [74]:
response_1 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_1["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy based on the requirements:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14  # RSI short period
        self.long_period = 28  # RSI long period
        self.rsi_lower_band = 30  # RSI lower band for buying
        self.rsi_upper_band = 70  # RSI upper band for selling
        self.is_long = False  # Flag to track if in a long position

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        if len(close_price_history) < self.long_period + 1:
            return []

        curr_rsi_short = rsi_short[-1]
        curr_rsi_

In [75]:
for document in response_1["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [76]:
response_2 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_2["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy implementation:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_upper_band = 70
        self.rsi_lower_band = 30

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band:
            # Buy Bitcoin
            return "Buy"
        elif curr_rsi_short > self.rsi_upper_band:
            # Sell Bitcoin
            return "Sell"
        else:
            return "H

In [77]:
for document in response_2["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045


In [78]:
response_3 = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response_3["answer"])

To create a strategy where you buy Bitcoin when its Relative Strength Index (RSI) drops below 30 and sell when it goes above 70, you can modify the existing code provided from Crypto Arsenal's documentation. Here's a simplified version of the strategy based on the requirements:

```python
import numpy as np
import talib

class RsiStrategy:
    def __init__(self):
        self.short_period = 14
        self.long_period = 28
        self.rsi_lower_band = 30
        self.rsi_upper_band = 70

    def trade(self, close_price_history):
        rsi_short = talib.RSI(np.array(close_price_history), self.short_period)
        rsi_long = talib.RSI(np.array(close_price_history), self.long_period)

        curr_rsi_short = rsi_short[-1]
        curr_rsi_long = rsi_long[-1]

        if curr_rsi_short < self.rsi_lower_band:
            # Buy Bitcoin
            return "Buy"
        elif curr_rsi_short > self.rsi_upper_band:
            # Sell Bitcoin
            return "Sell"
        else:
          

In [79]:
for document in response_3["context"]:
    print(f"Source: {document.metadata['source']}, Start_index: {document.metadata['start_index']}")
    #print(document.page_content)
    #print('#############################################################')

Source: RSI-en.py, Start_index: 2626
Source: RSI-en.py, Start_index: 1302
Source: RSI-en.py, Start_index: 0
Source: Bollinger_Bands-en.py, Start_index: 2045
